每次都要重新下载依赖

In [ ]:
%%sh
pip3 install -r ./requirements.txt

如果说/data目录下的文件包含有下图中的全部, 那就跳过下面这行指令.
<center><img src="https://naturalifica.oss-cn-nanjing.aliyuncs.com/~/Users/wuchentian/SoloLearning/Blog/source/imgs/image-20231105215016698.png" style="zoom:50%"></center>

In [ ]:
%%sh
cd ./data/raw && python3 -B raw.py && cd ../..

In [ ]:
%%sh
MY_PYTHON="python3 -B"
MNIST_ROTA="--n_layers 2 --n_hiddens 100 --data_path data/ --save_path results/ --batch_size 10 --log_every 100 --samples_per_task 1000 --data_file mnist_rotations.pt    --cuda no  --seed 0"
MNIST_PERM="--n_layers 2 --n_hiddens 100 --data_path data/ --save_path results/ --batch_size 10 --log_every 100 --samples_per_task 1000 --data_file mnist_permutations.pt --cuda no  --seed 0"
CIFAR_100i="--n_layers 2 --n_hiddens 100 --data_path data/ --save_path results/ --batch_size 10 --log_every 100 --samples_per_task 2500 --data_file cifar100.pt           --cuda yes --seed 0"

cd ./data

$MY_PYTHON mnist_permutations.py \
	--o mnist_permutations.pt \
	--seed 0 \
	--n_tasks 20
echo "Generating $/data/mnist_permutations.pt"

$MY_PYTHON mnist_rotations.py \
	--o mnist_rotations.pt\
	--seed 0 \
	--min_rot 0 \
	--max_rot 180 \
	--n_tasks 20
echo "Generating $/data/mnist_rotations.pt"	

$MY_PYTHON cifar100.py \
	--o cifar100.pt \
	--seed 0 \
	--n_tasks 20
echo "Generating $/data/cifar100.pt"	

cd ..

# model "single"
echo "Running single model"
$MY_PYTHON main.py $MNIST_ROTA --model single --lr 0.003
echo "Fishishing MNIST_ROTA --model single --lr 0.003"
$MY_PYTHON main.py $MNIST_PERM --model single --lr 0.03
echo "Fishishing MNIST_PERM --model single --lr 0.03"
$MY_PYTHON main.py $CIFAR_100i --model single --lr 1.0
echo "Fishishing CIFAR_100i --model single --lr 1.0"
 
# model "independent"
echo "Running independent model"
$MY_PYTHON main.py $MNIST_ROTA --model independent --lr 0.1  --finetune yes 
echo "Fishishing MNIST_ROTA --model independent --lr 0.1  --finetune yes"
$MY_PYTHON main.py $MNIST_PERM --model independent --lr 0.03 --finetune yes 
echo "Fishishing MNIST_PERM --model independent --lr 0.03 --finetune yes"
$MY_PYTHON main.py $CIFAR_100i --model independent --lr 0.3  --finetune yes 
echo "Fishishing CIFAR_100i --model independent --lr 0.3  --finetune yes"

# model "multimodal"
echo "Running multimodal model"
$MY_PYTHON main.py $MNIST_ROTA  --model multimodal --lr 0.1
echo "Fishishing MNIST_ROTA  --model multimodal --lr 0.1"
$MY_PYTHON main.py $MNIST_PERM  --model multimodal --lr 0.1
echo "Fishishing MNIST_PERM  --model multimodal --lr 0.1"

# model "EWC"
echo "Running EWC model"
$MY_PYTHON main.py $MNIST_ROTA --model ewc --lr 0.01 --n_memories 1000 --memory_strength 1000
echo "Fishishing MNIST_ROTA --model ewc --lr 0.01 --n_memories 1000 --memory_strength 1000"
$MY_PYTHON main.py $MNIST_PERM --model ewc --lr 0.1  --n_memories 10   --memory_strength 3
echo "Fishishing MNIST_PERM --model ewc --lr 0.1  --n_memories 10   --memory_strength 3"
$MY_PYTHON main.py $CIFAR_100i --model ewc --lr 1.0  --n_memories 10   --memory_strength 1
echo "Fishishing CIFAR_100i --model ewc --lr 1.0  --n_memories 10   --memory_strength 1"

# model "iCARL"
echo "Running iCARL model"
$MY_PYTHON main.py $CIFAR_100i --model icarl --lr 1.0 --n_memories 1280 --memory_strength 1
echo "Fishishing CIFAR_100i --model icarl --lr 1.0 --n_memories 1280 --memory_strength 1"

# model "GEM"
echo "Running GEM model"
$MY_PYTHON main.py $MNIST_ROTA --model gem --lr 0.1 --n_memories 256 --memory_strength 0.5
echo "Fishishing MNIST_ROTA --model gem --lr 0.1 --n_memories 256 --memory_strength 0.5"
$MY_PYTHON main.py $MNIST_PERM --model gem --lr 0.1 --n_memories 256 --memory_strength 0.5
echo "Fishishing MNIST_PERM --model gem --lr 0.1 --n_memories 256 --memory_strength 0.5"
$MY_PYTHON main.py $CIFAR_100i --model gem --lr 0.1 --n_memories 256 --memory_strength 0.5
echo "Fishishing CIFAR_100i --model gem --lr 0.1 --n_memories 256 --memory_strength 0.5"

# plot results
cd results/
echo "Ploting results"
$MY_PYTHON plot_results.py
cd ..